In [1]:
!pip install pandas

In [2]:
import pandas as pd
import numpy as np
import requests # library to handle requests
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Solving environment: ...working... done

# All requested packages already installed.



In [18]:
df = pd.read_csv("https://raw.githubusercontent.com/ninjananjo/Clothing-Store-Competition/master/StoreLocations.csv")
df.shape

(8, 56)

In [19]:
df = df[['TOWN','NAME','ID','COUNTRY','Address1','Postcode','Host']]
df = df[df['ID'].str.startswith("TS")]
df.head()

,TOWN,NAME,ID,COUNTRY,Address1,Postcode,Host
2,LONDON (OXFORD CIRCUS),OXFORD CIRCUS,TS0001,ENG,214 Oxford Street,W1W 8LG,TS0001
3,LONDON (CITY),CHEAPSIDE,TS0873,ENG,Unit Msu1,EC2V 6AH,TS0873
6,MANCHESTER (TRAFFORD C),TRAFFORD CENTRE,TS1012,ENG,130-134 Regent Crescent,M17 8AF,TS1012


In [39]:
#Add lontitude and lattitude data
addresses = df.loc[:,'Postcode']
addresses_alt = df.loc[:,'NAME']
latitude = []
longitude = []

for address, address_alt in zip(addresses, addresses_alt):
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(address+", UK")
    location_alt = geolocator.geocode(address_alt+", UK")
    try:
        #print('a')
        latitude.append(location.latitude)
        longitude.append(location.longitude)
        
    
    except:
        
        try:
            #print('b')
            latitude.append(location_alt.latitude)
            longitude.append(location_alt.longitude)
            
        except:    
            #print('c')
            latitude.append(0)
            longitude.append(0)
            
        
df.loc[:,'Latitude']= latitude
df.loc[:,'Longitude']= longitude

a
a
a
b


In [41]:
#remove regions that location information was not available
df = df[(df['Latitude']!=0)| (df['Longitude']!=0)]
print(df.shape[0])
df

3


,TOWN,NAME,ID,COUNTRY,Address1,Postcode,Host,Latitude,Longitude
2,LONDON (OXFORD CIRCUS),OXFORD CIRCUS,TS0001,ENG,214 Oxford Street,W1W 8LG,TS0001,51.515534,-0.141352
3,LONDON (CITY),CHEAPSIDE,TS0873,ENG,Unit Msu1,EC2V 6AH,TS0873,51.514098,-0.094914
6,MANCHESTER (TRAFFORD C),TRAFFORD CENTRE,TS1012,ENG,130-134 Regent Crescent,M17 8AF,TS1012,53.462559,-2.346119


In [62]:
CLIENT_ID = 'QKP4KVQQZPVXMOKNIXHGQAT1E2IJZCHQPFB4XJGP3MC0CFUO' # your Foursquare ID
CLIENT_SECRET = '1N5BIRBR1EA5CO1QFMNHU1241SOFM5KJLLQRJGMDC0NOYWPH' # your Foursquare Secret
VERSION = '20190501' # Foursquare API version
LIMIT = 400 # limit of number of venues returned by Foursquare API
radius = 10000 # define radius
categoryId = '4bf58dd8d48988d1f2941735'

In [63]:

#What are the most popular places nearby?

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    
    region_list = []
    
    for name, lat, lng in zip(names, latitudes, longitudes):
                    
        # create the API request URL
        #url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        #url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(    
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            categoryId)
        
        #print(url) 
        
        # make the GET request
        results = requests.get(url).json()["response"]
        
        #flatten JSON file
        resultsVen = json_normalize(results['venues'])
        resultsCat = json_normalize(data=results['venues'], record_path=['categories'], record_prefix='cat_', errors='ignore')
        resultsVenCat = pd.concat([resultsVen, resultsCat], axis=1)
        resultsVenCat = resultsVenCat[['id','name','location.distance','location.formattedAddress','location.lat','location.lng','cat_name']]
        
        #Add region name and average price 
        for i in range(0,resultsVen.shape[0]):
            region_list.append(name)
                    
        region_list = pd.DataFrame(region_list, columns=['Store ID'])
        
        nearbyVenues = pd.concat([region_list, resultsVenCat], axis=1)

    return(nearbyVenues)

In [64]:
regions_venues = pd.DataFrame()

#request venue information for each region and store in regions_venues
for region in df['ID'].unique():
    region_venues = df[df['ID']==region]
    regions_venues = regions_venues.append((getNearbyVenues(names=region_venues['ID'],
                                                  latitudes=region_venues['Latitude'],
                                                  longitudes=region_venues['Longitude'],
                                                  radius=radius)),ignore_index=True)
    
#rename columns
regions_venues.rename(columns={'id':'Venue ID',
                          'name':'Venue Name',
                          'location.distance':'Venue Distance',
                          'location.formattedAddress':'Venue Address',
                          'location.lat':'Venue Lat',
                          'location.lng':'Venue Lng',
                          'cat_name':'Venue Category'}, inplace=True)

In [65]:
pd.set_option('display.max_rows', 500)
print(regions_venues.shape[0])
regions_venues.head()

145


,Store ID,Venue ID,Venue Name,Venue Distance,Venue Address,Venue Lat,Venue Lng,Venue Category
0,TS0001,4ac518f0f964a520afad20e3,NikeTown,33,"[236 Oxford St, London, Greater London, W1C 1D...",51.515488,-0.141824,Sporting Goods Shop
1,TS0001,4ac518f1f964a520ecad20e3,JD Sports,127,"[197-203 Oxford Street, West End, Greater Lond...",51.515239,-0.139567,Sporting Goods Shop
2,TS0001,51d86b1d498e44075e0a7840,Foot Locker,439,"[109 Oxford Street, London, Greater London, W1...",51.515890,-0.135038,Sporting Goods Shop
3,TS0001,5ca5b84a9d7468003917baa1,Up & Running,2405,"[72-74 Fleet Street, London, Greater London, E...",51.514142,-0.106705,Sporting Goods Shop
4,TS0001,5804fc2b38fa33f7c3387dae,New Balance,175,"[287 - 291 Oxford Street, London, Greater Lond...",51.515116,-0.143791,Sporting Goods Shop


In [66]:
jdsports = regions_venues[regions_venues['Venue Name']=='JD Sports']
jdsports

,Store ID,Venue ID,Venue Name,Venue Distance,Venue Address,Venue Lat,Venue Lng,Venue Category
1,TS0001,4ac518f1f964a520ecad20e3,JD Sports,127,"[197-203 Oxford Street, West End, Greater Lond...",51.515239,-0.139567,Sporting Goods Shop
9,TS0001,4fbf51d3e4b07c0ab040c868,JD Sports,11112,"[Unit C The Mall 33 Selbourne Walk, Walthamsto...",51.583132,-0.023231,Sporting Goods Shop
19,TS0001,5a8ecd3d5a2c9117b2eca1d4,JD Sports,9868,"[53 Tooting High Street, Tooting, Greater Lond...",51.428388,-0.167446,Sporting Goods Shop
54,TS0873,4fbf51d3e4b07c0ab040c868,JD Sports,9147,"[Unit C The Mall 33 Selbourne Walk, Walthamsto...",51.583132,-0.023231,Sporting Goods Shop
71,TS0873,4ac518f1f964a520ecad20e3,JD Sports,3095,"[197-203 Oxford Street, West End, Greater Lond...",51.515239,-0.139567,Sporting Goods Shop
100,TS1012,4edd380cbe7be2833b142d45,JD Sports,482,"[48 Regents Crescent, Unit L12, Trafford Centr...",53.466613,-2.348693,Sporting Goods Shop
102,TS1012,5a8eceff67af3a74d388fc4b,JD Sports,10761,"[Unit 23 The Birtles Sc, Wythenshawe, Greater ...",53.379649,-2.262698,Sporting Goods Shop
104,TS1012,4b7ec528f964a52010fe2fe3,JD Sports,7233,"[16-18 Arndale Centre, Barbarolli Mall, Manche...",53.483574,-2.242812,Sporting Goods Shop
110,TS1012,5a4545b8c0f16320555bd027,JD Sports,5603,"[Unit G, Regent Retail Park, Salford, M5 4DE, ...",53.477943,-2.265591,Sporting Goods Shop
115,TS1012,4cce650a97d0224b3ba86bb8,JD Sports,14334,"[37 Market Place, Bolton, Lancashire, BL1 2AL,...",53.581322,-2.429816,Sporting Goods Shop
